Conclusion: 
- Dans ce notebook, on regarde l'impact de : 
    - la durée de vie 
    - l'odomètre 
    - la chimie
    - le type de charge 
    - la température -> pas de conclusion intéressante

On stocke les résultats propre ici : https://www.notion.so/bib-batteries/Etude-des-param-tres-de-d-gradations-des-batteries-1a72de3b75c78054bdb1e9ae87c6670d?pvs=4

In [ ]:
import pandas as pd
import plotly.express as px
import numpy as np
from dateutil.relativedelta import relativedelta


In [ ]:
from  transform.processed_tss.ProcessedTimeSeries import TeslaProcessedTimeSeries
from transform.raw_results.config import *
from transform.processed_results.main import get_processed_results

# Load data

In [ ]:
results = get_processed_results('tesla').dropna(subset='soh')

In [ ]:
from core.sql_utils import *
engine = get_sqlalchemy_engine()
con = engine.connect()

with engine.connect() as connection:
    dbeaver_df = pd.read_sql(text("""SELECT * FROM vehicle_data vd
            join vehicle v
            on v.id = vd.vehicle_id
            join vehicle_model vm 
            on vm.id = v.vehicle_model_id
            join battery b 
            on b.id=vm.battery_id
            WHERE vm.model_name like '%model%';"""), con)

battery_chemistry_df = dbeaver_df.groupby('vin', as_index=False).agg(
    battery_chemistry=("battery_chemistry", 'first'),
    start_date=('start_date', 'first'),
    tesla_code=('version', 'first')

    
)

In [ ]:
results

In [ ]:
results = results.merge(battery_chemistry_df, how='left', on='vin')

## Tesla battery label & chemistry

In [ ]:
# build life battery in month
def compute_life_battery(start_date, last_date):
    try:
        years = relativedelta(start_date, last_date).years
        months = relativedelta(start_date, last_date).months
        return years*12 + months
    except:
        return 0
    
results = results.merge(results.groupby('vin', as_index=False, observed=False).agg(life_battery=('date', 'max')), on='vin')
results['life_battery'] = results.apply(lambda x: compute_life_battery(x['life_battery'], x['start_date']), axis=1)

In [ ]:
# build interval for vehicles odometer
results['odometer_interval'] = results['odometer'].apply(lambda x: "< 50k" if x < 50_000 
                                          else "50k-80k"  if x >= 50_000 and x < 80_000
                                          else "80-120" if x >= 80_000 and x <120_000
                                          else "120k >")

In [ ]:
vin_soh = results.groupby('vin', as_index=False, observed=True).agg(
    chemistry= ('battery_chemistry','first'), 
    soh_mean=('soh', 'mean'),
    soh_median=('soh', 'median'),
    odometer_interval = ('odometer_interval',"last"),
    odometer = ('odometer', "max"),
    life_battery = ('life_battery','max'),
    tesla_code = ('tesla_code', 'first'),
    )

# EDA

## Diff of the impact beetween odometer and life battery on SoH

In [ ]:
px.imshow(vin_soh[['soh_median', 'odometer', 'life_battery']].corr()[['soh_median']], text_auto=True, color_continuous_scale='viridis')

In [ ]:
px.scatter(vin_soh[vin_soh['life_battery']>0][['odometer', 'soh_median', 'life_battery', 'chemistry', 'odometer_interval']],  
           x='life_battery', y='soh_median', title="Impact of battery life on SoH", trendline='ols', color='odometer')

In [ ]:
px.scatter(vin_soh[vin_soh['life_battery']>0][['odometer', 'soh_median', 'life_battery', 'chemistry']],  x='odometer', y='soh_median', title="Impact of mileage on SoH", trendline='ols', color='life_battery')

## Chemistry impact 

The chimistry impact the SoH but we want to know **_how much ?_**

Chemistry repartition by vin

In [ ]:
vin_soh.vin.nunique()

In [ ]:
vin_soh.value_counts('chemistry').sum()

In [ ]:
vin_soh.dropna(subset=['soh_mean']).value_counts('chemistry', normalize=True)

In [ ]:
chemistry_rep_v = pd.DataFrame(vin_soh.value_counts(['chemistry'])).reset_index().rename(columns={'count':'total_vehicles_chemistry'})
chemistry_odo_rep_v = pd.DataFrame(vin_soh.value_counts(['chemistry', 'odometer_interval']).reset_index()).rename(columns={'count':'total_vehicles_chemistry_odometer'})
rep = chemistry_rep_v.merge(chemistry_odo_rep_v,  on='chemistry')
rep['proportion_chemistry'] = (rep['total_vehicles_chemistry_odometer'] / rep['total_vehicles_chemistry'] ).round(2)
rep

In [ ]:
px.scatter(vin_soh, x='odometer', y='soh_median', color="chemistry", hover_data="tesla_code")

On constate un nueage tout droit au niveau de .9 de soh qui correspond au MT336. 

In [ ]:
vin_soh.groupby(["odometer_interval", "chemistry"], as_index=False)[['soh_median', 'soh_mean']].mean().sort_values(["odometer_interval", 'soh_median'], ascending=False)

In [ ]:
px.scatter(vin_soh[(vin_soh['chemistry']=='LFP') & (vin_soh['tesla_code']!='MT336')], x='odometer', y='soh_median', color="tesla_code", hover_data="tesla_code", title='SoH distribution for LFP chemistry')

In [ ]:
vin_soh[vin_soh['tesla_code'].isin(['MT336' ])].vin.nunique()

In [ ]:
# mean over the median and the mean by vin 
vin_soh[~vin_soh['tesla_code'].isin(['MT336' ])].groupby([ 'odometer_interval', 'chemistry'])[['soh_median', 'soh_mean']].mean().sort_index()

**Général**
Le SOH diminue avec l’augmentation du kilométrage pour toutes les chimies de batteries (LFP, NCA, NMC).
Les batteries LFP se dégradent moins rapidement que les NCA et NMC.
Les NMC conservent généralement un bon SOH, mais restent légèrement derrière le LFP en matière de rétention.
Les NCA se dégradent plus rapidement, surtout au-delà de 80 000 km.

**< 50 000 km**

SOH le plus élevé pour toutes les chimies.

LFP : ~0,979

NMC : ~0,973

NCA : ~0,943 (déjà plus faible que les autres chimies).

**50 000 - 80 000 km**

Dégradation modérée pour toutes les batteries.

LFP : ~0,973

NMC : ~0,945

NCA : ~0,941

**80 000 - 120 000 km**

Le LFP conserve un bon SOH, mieux que les autres chimies.

LFP : ~0,959

NMC : ~0,933

NCA : ~0,927

**120 000+ km**

Dégradation plus marquée, surtout pour le NCA.

LFP : ~0,950 (meilleure longévité).

NMC : ~0,921

NCA : ~0,908 (se dégrade le plus).

**Conclusion**

Le LFP est le plus performant en matière de rétention du SOH sur l’ensemble des kilométrages.

Le NMC suit de près mais se dégrade légèrement plus vite que le LFP.

Le NCA se dégrade le plus rapidement, surtout après 80 000 km.

Par ailleurs, la répartition des véhicules par kilométrage n’est pas homogène :

70 % des véhicules LFP ont parcouru moins de 50 000 km.

Seuls 5 % des véhicules NCA ont moins de 50 000 km, ce qui peut influencer les moyennes observées.